In [1]:
import pandas as pd
import os
import json
import requests

In [149]:
#endpoint = "http://api.worldbank.org/v2/indicator?format=json"
#endpoint = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.MKTP.KD.ZG?format=json"
#endpoint = "http://api.worldbank.org/v2/countries?format=json"
#endpoint = "http://api.worldbank.org/v2/sources/57/country/all/series/NY.GDP.MKTP.PP.CD/time/all/data?format=json"
endpoint = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD?format=json&per_page=16104"

response = requests.get(endpoint)

print(response)

<Response [200]>


In [150]:
response = response.json()

In [151]:
response[1][0]["country"]["value"]

'Arab World'

In [152]:
print(json.dumps(response, indent=4))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [153]:
test = pd.DataFrame(response[1])
test

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': '1A', 'value': 'Arab World'}",ARB,2020,NaN,,,1
1,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': '1A', 'value': 'Arab World'}",ARB,2019,6580.056257,,,1
2,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': '1A', 'value': 'Arab World'}",ARB,2018,6603.351190,,,1
3,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': '1A', 'value': 'Arab World'}",ARB,2017,6102.211067,,,1
4,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': '1A', 'value': 'Arab World'}",ARB,2016,6097.239025,,,1
...,...,...,...,...,...,...,...,...
16099,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,1964,281.558440,,,1
16100,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,1963,277.479715,,,1
16101,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,1962,276.688781,,,1
16102,"{'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per cap...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,1961,280.828951,,,1


In [154]:
new = pd.DataFrame()
new["country"] = ""
new["countryiso3code"] = ""
new["date"] = ""
new["value"] = ""
new["indicator"] = ""
new["description"] = ""

for idx, row in test.iterrows():
    
    new.loc[idx,"country"] = dict(row["country"])["value"]
    new.loc[idx,"countryiso3code"] = row["countryiso3code"]
    new.loc[idx,"date"] = row["date"]
    new.loc[idx,"value"] = row["value"]
    new.loc[idx,"indicator"] = dict(row["indicator"])["id"]
    new.loc[idx,"description"] = dict(row["indicator"])["value"]
        
    #print(f'country: {dict(row["country"])["value"]} - indicator : {dict(row["indicator"])["id"]} - value : {row["value"]}')
    
new

,country,countryiso3code,date,value,indicator,description
0,Arab World,ARB,2020,NaN,NY.GDP.PCAP.CD,GDP per capita (current US$)
1,Arab World,ARB,2019,6580.06,NY.GDP.PCAP.CD,GDP per capita (current US$)
2,Arab World,ARB,2018,6603.35,NY.GDP.PCAP.CD,GDP per capita (current US$)
3,Arab World,ARB,2017,6102.21,NY.GDP.PCAP.CD,GDP per capita (current US$)
4,Arab World,ARB,2016,6097.24,NY.GDP.PCAP.CD,GDP per capita (current US$)
...,...,...,...,...,...,...
16099,Zimbabwe,ZWE,1964,281.558,NY.GDP.PCAP.CD,GDP per capita (current US$)
16100,Zimbabwe,ZWE,1963,277.48,NY.GDP.PCAP.CD,GDP per capita (current US$)
16101,Zimbabwe,ZWE,1962,276.689,NY.GDP.PCAP.CD,GDP per capita (current US$)
16102,Zimbabwe,ZWE,1961,280.829,NY.GDP.PCAP.CD,GDP per capita (current US$)
